In [1]:
%matplotlib inline
from pathlib import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import sys 

In [2]:
sys.path.append('../../fastai/')

In [3]:
from fastai.conv_learner import *

In [4]:
path = list(Path('../data/ds_bowl_2018/').iterdir())

In [6]:
path

[PosixPath('../data/ds_bowl_2018/stage1_train_labels.csv'),
 PosixPath('../data/ds_bowl_2018/stage1_train'),
 PosixPath('../data/ds_bowl_2018/stage2_test_final'),
 PosixPath('../data/ds_bowl_2018/stage1_solution.csv')]

In [7]:
trn_dirs = list(path[1].iterdir())

In [9]:
solutions = pd.read_csv(path[-1]) # solutions for stage1 test

In [10]:
def run_length_decode(rle, H, W, fill_value=255):

    mask = np.zeros((H * W), np.uint8)
    rle = np.array([int(s) for s in rle.split(' ')]).reshape(-1, 2)
    for r in rle:
        start = r[0]-1
        end = start + r[1]
        mask[start : end] = fill_value
    mask = mask.reshape(W, H).T # H, W need to swap as transposing.
    return mask

In [13]:
stage1_solution = pd.read_csv(path[-1])
stage1_train_labels = pd.read_csv(path[0])

In [21]:
stage1_all_labels = pd.concat([stage1_train_labels, stage1_solution.iloc[:, :2]])

### Create one masks

In [22]:
for trn_dir in trn_dirs:
    dirname = trn_dir.name
    os.makedirs(trn_dir/'one_mask/', exist_ok=True) # create masks dir

    test_case = stage1_all_labels[stage1_all_labels.ImageId == dirname].reset_index()
    rles = test_case['EncodedPixels']
    H, W = open_image(list((trn_dir/'images').iterdir())[0]).shape[:-1]
    
    decoded_sum = 0
    for i, rle in enumerate(rles):
        decoded = run_length_decode(rle, H, W, 255)
        decoded_sum += decoded
    print(len(rles))
    plt.imsave(str(trn_dir/'one_mask') + '/one_mask.png', decoded_sum)

21
24
75
14
55
12
16
44
44
27
9
20
56
64
42
19
37
8
15
70
8
28
59
11
10
27
44
41
89
8
5
12
101
10
22
130
11
158
28
44
21
92
133
21
10
20
50
5
53
134
25
18
29
106
8
39
12
25
136
70
8
30
10
99
65
37
6
25
57
369
29
13
183
36
34
76
66
5
9
38
12
19
36
14
23
15
28
22
12
7
10
65
48
13
56
36
111
25
141
23
23
16
25
34
14
54
6
3
13
14
18
31
10
24
10
14
59
125
7
6
7
23
9
53
23
14
62
18
22
11
170
30
11
11
257
62
8
107
66
6
13
11
10
44
23
49
2
98
101
29
12
14
25
37
30
28
3
22
40
100
24
46
15
22
61
78
64
8
12
29
40
10
8
26
5
5
23
64
25
16
72
9
26
17
27
18
16
98
7
17
78
132
106
39
38
102
9
71
155
28
54
20
73
58
6
15
45
13
28
25
2
39
37
7
10
37
17
50
23
25
36
7
119
52
90
84
49
94
31
31
329
43
24
157
78
20
272
9
10
168
9
68
6
42
50
40
98
2
28
22
35
21
86
7
25
146
18
38
11
112
81
29
14
20
15
28
58
48
83
38
61
198
11
375
4
106
21
11
146
25
17
88
20
18
22
28
18
27
22
54
17
124
16
30
21
31
82
12
69
191
20
1
16
33
23
73
22
36
289
196
20
12
138
28
8
29
27
6
89
47
60
32
119
97
29
31
8
74
17
22
30
12
10
20
20


### Create Individual Masks

In [27]:
for trn_dir in trn_dirs:
    dirname = trn_dir.name
    try:
        os.makedirs(trn_dir/'masks/', exist_ok=False) # create masks dir

        test_case = stage1_all_labels[stage1_all_labels.ImageId == dirname].reset_index()
        rles = test_case['EncodedPixels']
        H, W = open_image(list((trn_dir/'images').iterdir())[0]).shape[:-1]

        for i, rle in enumerate(rles):
            decoded = run_length_decode(rle, H, W, 255)
            plt.imsave((str(trn_dir/'masks') + f'/mask{i}.png'), decoded)
    except:pass

### Check

In [23]:
for trn_dir in trn_dirs:
    if open_image(str(trn_dir/'one_mask/one_mask.png')) is None:
        print(trn_dir.name)